텍스트 분류
=====
영화 시놉시스로 장르 예측하기
-----
지난 감성분석(이진분류)에 이어 이번에는 다중 클래스 텍스트 분류를 해보겠습니다.<br>
이번에 살펴볼 모델은 나이브 베이즈 분류기와 서포트 벡터 머신입니다.

데이터는 지난 텍스트 표현 실습에서 활용했던 영화 시놉시스 데이터를 장르까지 다시 크롤링한 데이터입니다.<br>
데이터 수를 많이 줄여서 OOM은 안뜰겁니다! <br>
해당 데이터의 특징은 이미 토픽(클래스)이 주어져 있다는 것입니다. 이 분류에 맞게 잘 판단하는지 확인해봅시다.

In [4]:
import pandas as pd

In [5]:
%ls

 C 드라이브의 볼륨: SSD
 볼륨 일련 번호: 786C-ABF7

 C:\Users\HONG\Desktop\HONGYP\BOAZ\New 텍스트마이닝\6주차 디렉터리

2020-08-27  오후 09:06    <DIR>          .
2020-08-27  오후 09:06    <DIR>          ..
2020-08-27  오후 07:19    <DIR>          .ipynb_checkpoints
2020-08-27  오후 09:06           170,162 10강_Word2Vec_배포용.ipynb
2020-08-27  오후 07:13            22,578 8zNRwsn.png
2020-08-26  오후 01:21            87,999 9강_텍스트 분류_배포용.ipynb
2020-08-27  오후 08:56        78,903,658 FastText.model
2020-08-27  오후 08:56       800,000,128 FastText.model.trainables.vectors_ngrams_lockf.npy
2020-08-27  오후 08:56       800,000,128 FastText.model.wv.vectors_ngrams.npy
2020-08-27  오후 07:16         3,845,115 word2vec ppt.pptx
2020-08-27  오후 07:25        28,594,626 네이버 영화 시놉시스 데이터 (토큰화 완료).csv
2020-08-26  오후 12:53        11,310,766 네이버 영화 시놉시스 데이터.csv
               9개 파일       1,722,935,160 바이트
               3개 디렉터리  184,888,369,152 바이트 남음


In [6]:
df = pd.read_csv('네이버 영화 시놉시스 데이터.csv')

In [7]:
df.drop(["Unnamed: 0", "Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1"], axis = 1, inplace = True)

In [8]:
df.head()

,영화제목,시놉시스,장르
0,증인,“목격자가 있어. 자폐아야” 신념은 잠시 접어두고 현실을 위해 속물이 되기로 마음...,드라마
1,청년경찰,의욕충만 경찰대생 기준(박서준) X 이론백단 경찰대생 희열(강하늘) 둘도 없는 친구...,액션
2,비긴 어게인,싱어송라이터인 ‘그레타’(키이라 나이틀리)는 남자친구 ‘데이브’(애덤 리바인)가 메...,코미디
3,신과함께-죄와 벌,"저승 법에 의하면, 모든 인간은 사후 49일 동안 7번의 재판을 거쳐야만 한다. 살...",드라마
4,아이 필 프리티,뛰어난 패션센스에 매력적인 성격이지만 통통한 몸매가 불만인 ‘르네’ 하아.. 예뻐지...,코미디


In [9]:
import re

def cleaning(x):
    return re.sub("[^\w\s]", " ", x)

In [10]:
df["시놉시스"] = df["시놉시스"].apply(cleaning)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7785 entries, 0 to 7784
Data columns (total 3 columns):
영화제목    7785 non-null object
시놉시스    7785 non-null object
장르      7785 non-null object
dtypes: object(3)
memory usage: 182.5+ KB


In [12]:
# 결측치 확인
df[df['시놉시스'].isnull()]

,영화제목,시놉시스,장르


In [13]:
df["장르"].unique()

array(['드라마', '액션', '코미디', '다큐멘터리', '공포/스릴러', '애니메이션', 'SF/판타지'],
      dtype=object)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB # 다중 Naive Bayes 분류기
from sklearn.metrics import accuracy_score

In [15]:
# 텍스트 전처리
import konlpy.tag

okt = konlpy.tag.Okt()
df['시놉시스_토큰'] = df['시놉시스'].apply(lambda x : okt.nouns(x)).apply(lambda x : ' '.join(x))
# 실행에 30분 이상 걸려서 전처리한 내용을 미리 가지고 왔습니다.

## Naive Bayes

In [16]:
# 데이터 스플릿(훈련 데이터와 테스트 데이터)
train_texts, test_texts, train_labels, test_labels = train_test_split(df['시놉시스_토큰'], df['장르'], test_size=0.3, random_state=0)

In [17]:
# TF-IDF 벡터화
vectorizer = TfidfVectorizer(min_df = 3, ngram_range = (1, 1))

In [18]:
features = vectorizer.fit_transform(train_texts)

In [19]:
# Naive Bayes 모델
nb = MultinomialNB(alpha = 0.01)

In [20]:
# Naive Bayes 모델 훈련 데이터로 학습
nb.fit(features, train_labels)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [21]:
# 테스트 데이터 벡터화
test_features = vectorizer.transform(test_texts)

In [22]:
# Naive Bayes 모델로 분류 예측하기
nb.predict(test_features)

array(['액션', '액션', '액션', ..., '드라마', '드라마', '드라마'], dtype='<U6')

In [23]:
# 분류 예측 결과 확인
pred_labels = nb.predict(test_features)
print('잘못 분류된 영화장르: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('정확도: %.2f' % accuracy_score(test_labels, pred_labels))

잘못 분류된 강의평: 554 out of 2336
정확도: 0.76


In [24]:
# 분류 예측 결과 DataFrame 확인
test_df = pd.DataFrame({'영화제목':df["영화제목"][test_texts.index], '시놉시스_토큰' : test_texts, '장르_예측' : pred_labels, "실제장르": test_labels})

test_df

,영화제목,시놉시스_토큰,장르_예측,실제장르
2476,킬 커맨드: 드론의 습격,미래 캡틴 버크 사령관 최 정예 해병 대원 외부 로부터 통신 단절 아무 위치 섬 일...,액션,액션
5178,맨 오브 타이치,태극권 위해 전사 패배 곧 죽음 택배 기사 일 타이거 타이거 후 첸 링쿵 태극권 계...,액션,액션
5585,팅커 벨 2: 팅커벨과 잃어버린 보물,전세계 사랑 요정 팅커벨 상상 요정 나라 픽시 할로우 흥미진진 팅커벨 모험 가을 추...,액션,액션
7276,골든 슬럼버,숲 도시 센다이 반미 성향 신임 총리 취임 퍼레이드 중 헬기 폭탄 폭발 국민 목격 ...,공포/스릴러,공포/스릴러
3554,하나레이 베이,사치 하와이 하나 레이 해변 아들 서핑 중 목숨 소식 그 후 십 년 동안 그녀 매년...,액션,드라마
200,비포 선라이즈,소르본느 대학생 셀린느 줄리 델피 분 부다페스트 할머니 가을 학기 개강 빠리 길이 ...,드라마,드라마
979,청춘 스케치,레이나 이제 갓 대학 졸업 사회 초년 생 청운 꿈 안고 텍사스 방송국 입사 프로그램...,코미디,코미디
2923,트릭,사람 숫자 시청률 드라마 국 예능 국 오늘 앞 사람 시한 부 남편 도준 그 아내 영...,드라마,드라마
1277,엘 불리: 요리는 진행 중,프랑스 레스토랑 평가 미슐랭 가이드 최고 등급 별 셋 영국 음식 전문 레스토랑 이 ...,코미디,다큐멘터리
4375,샤크,무소 불위 절대 권력 막강 카리스마 상어 대부 돈 리노 그 말 고민 상어 본성 정도...,액션,액션


## SVC

In [25]:
from sklearn.svm import SVC

In [26]:
# SVC 불러오기
svm = SVC(C = 10, kernel = "rbf", gamma = 0.5)

In [27]:
# SVC 모델 훈련 데이터로 학습
svm.fit(features, train_labels)

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.5, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [28]:
# 분류 예측 결과 확인
pred_labels = svm.predict(test_features)
print('잘못 분류된 영화장르: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
print('정확도: %.2f' % accuracy_score(test_labels, pred_labels))

잘못 분류된 강의평: 488 out of 2336
정확도: 0.79


In [29]:
# 분류 예측 결과 DataFrame 확인
test_df = pd.DataFrame({'영화제목':df["영화제목"][test_texts.index], '시놉시스_토큰' : test_texts, '장르_예측' : pred_labels, "실제장르": test_labels})

test_df

,영화제목,시놉시스_토큰,장르_예측,실제장르
2476,킬 커맨드: 드론의 습격,미래 캡틴 버크 사령관 최 정예 해병 대원 외부 로부터 통신 단절 아무 위치 섬 일...,액션,액션
5178,맨 오브 타이치,태극권 위해 전사 패배 곧 죽음 택배 기사 일 타이거 타이거 후 첸 링쿵 태극권 계...,액션,액션
5585,팅커 벨 2: 팅커벨과 잃어버린 보물,전세계 사랑 요정 팅커벨 상상 요정 나라 픽시 할로우 흥미진진 팅커벨 모험 가을 추...,액션,액션
7276,골든 슬럼버,숲 도시 센다이 반미 성향 신임 총리 취임 퍼레이드 중 헬기 폭탄 폭발 국민 목격 ...,공포/스릴러,공포/스릴러
3554,하나레이 베이,사치 하와이 하나 레이 해변 아들 서핑 중 목숨 소식 그 후 십 년 동안 그녀 매년...,액션,드라마
200,비포 선라이즈,소르본느 대학생 셀린느 줄리 델피 분 부다페스트 할머니 가을 학기 개강 빠리 길이 ...,드라마,드라마
979,청춘 스케치,레이나 이제 갓 대학 졸업 사회 초년 생 청운 꿈 안고 텍사스 방송국 입사 프로그램...,코미디,코미디
2923,트릭,사람 숫자 시청률 드라마 국 예능 국 오늘 앞 사람 시한 부 남편 도준 그 아내 영...,코미디,드라마
1277,엘 불리: 요리는 진행 중,프랑스 레스토랑 평가 미슐랭 가이드 최고 등급 별 셋 영국 음식 전문 레스토랑 이 ...,액션,다큐멘터리
4375,샤크,무소 불위 절대 권력 막강 카리스마 상어 대부 돈 리노 그 말 고민 상어 본성 정도...,액션,액션
